In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import pandas as pd
import torch


In [ ]:
from smc01.postprocessing.dataset import SMCParquetDataset
from smc01.postprocessing.model.attention import SMCAttentionModel
from smc01.postprocessing.transform import NormalizeDataframe
from smc01.postprocessing.util import concat_collate_fn

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('train', [
        'experiment=emos_gdps_metar',
        '+experiment.dataset.step_id=12',
        'experiment.dataset.transform.transforms.1.add_metadata_features=True'
    ])

In [ ]:
cfg.experiment.dataset

In [ ]:
dataset = hydra.utils.instantiate(cfg.experiment.dataset)

In [ ]:
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=cfg.num_workers,
)

In [ ]:
dataset[0]['features'].shape

In [ ]:
m = SMCAttentionModel(len(dataset.stations), 10, 1, 128, 128)
m = m.cuda()
optimizer = torch.optim.Adam(m.parameters(), lr=1e-4)

In [ ]:
for b in loader:
    b = {k: b[k].cuda() for k in b}
    obs = b['obs']
    
    prediction = m(b)
    rmse = torch.sqrt(torch.square(obs - prediction).mean())
    rmse.backward()
    print(rmse)
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
m.embedding.